## ICC

In [2]:
import glob     # Import glob to easily loop over files
import pympi    # Import pympi to work with elan files
import os
import pprint as pp
import pandas as pd
import numpy as np
import pingouin as pg

# data = pd.read_csv("tt_ab_reli.csv", index_col=0)
data = pd.read_csv("tt_ab_reli.csv")
subjects = [1, 10, 18, 52, 57, 59, 164, 1001, 1004]
columns = ['sub','rater','rating']
#d = data.replace(["V", "CVCV","VCV", "CV", "VCVC", "VC", "CVC"], ["V","V","V","V","V","V","V"])
print(data.columns)
result = pd.DataFrame(columns=columns)
for sub in subjects:
    cols = ["sub", "rater", "rating"]
    df = pd.DataFrame(0, index=np.arange(2), columns= cols)
    df["sub"] = sub
    df["rater"] = ["original", "reli"]
    

    sub_frame = data[(data["sub"] == sub)&(data["tier"] == "Caregiver voc")]
#     sub_frame = data[(data["sub"] == sub)&(data["tier"] == "Infant voc")]
#     sub_frame = data[(data["sub"] == sub)]
    rating1 = len(sub_frame[sub_frame["reli"] == "original"])
    rating2 = len(sub_frame[sub_frame["reli"] == "reli"])
    df["rating"] = [rating1, rating2]
    result = result._append(df)

result["rating"] = pd.to_numeric(result["rating"])
print(result)
icc = pg.intraclass_corr(data = result, targets="sub", raters="rater", ratings = "rating")
icc = icc.set_index('Type')
icc = icc.loc["ICC3k","ICC"]
print(icc)

Index(['sub', 'onset', 'offset', 'cat', 'tier', 'reli'], dtype='object')
    sub     rater  rating
0     1  original     175
1     1      reli     184
0    10  original      85
1    10      reli      81
0    18  original      26
1    18      reli      52
0    52  original      63
1    52      reli      47
0    57  original      21
1    57      reli      12
0    59  original      42
1    59      reli       1
0   164  original      17
1   164      reli      20
0  1001  original      33
1  1001      reli      36
0  1004  original      69
1  1004      reli      71
0.9675780879141745


## Cohen's kappa

In [3]:
import math
import warnings
import numpy as np
import pandas as pd
from scipy import stats
from statistics import mean
warnings.filterwarnings('ignore')
import scipy.stats
import os
from sklearn.metrics import cohen_kappa_score

def reli_cal(data, subjects, reli1, reli2, label):
    
    data = pd.DataFrame(data=data)
    data["onset"] = data["onset"]*29.97
    data["offset"] = data["offset"]*29.97
    data.onset = data.onset.round(0)
    data.offset = data.offset.round(0)
    data["cat"] = data["cat"]
    print(data)

    kappa = []
    for sub in subjects:
        cols = ["sub", "coder1", "coder2"]
        df = pd.DataFrame(0, index=np.arange(12*60*29.97), columns= cols)
        df["sub"] = sub
    
        sub_frame = data[data["sub"] == sub] 
        for i, row in sub_frame.iterrows():
            if (row["reli"] == reli1):
                on = float(row["onset"])
                off = float(row["offset"])
                df.loc[on:off,"coder1"] = "A"
            if (row["reli"] == reli2):
                on = float(row["onset"])
                off = float(row["offset"])
                df.loc[on:off,"coder2"] = "A"
            df = df.replace(0, "None")
    
        labeler1 = df["coder1"].astype(str)
        labeler2 = df["coder2"].astype(str)
        ck = cohen_kappa_score(labeler1, labeler2,
                  labels= label)
        kappa.append(ck)
    print(kappa)

    print("Mean kappa value is", np.mean(kappa))

In [4]:
data = pd.read_csv("tt_ab_reli.csv")
# d = data.replace(["V", "CVCV","VCV", "CV", "VCVC", "VC", "CVC"], ["V","V","V","V","V","V","V"])
subjects = [1, 10, 18, 52, 57, 59, 164, 1001, 1004]
reli1 = "original"
reli2 = "reli"
label = ["A", "None"] 
reli_cal(data, subjects, reli1, reli2, label) 

       sub    onset   offset  \
0        1     23.0     51.0   
1        1     24.0     51.0   
2        1     51.0     78.0   
3        1     51.0     80.0   
4        1     78.0    138.0   
...    ...      ...      ...   
3505  1004  18088.0  18140.0   
3506  1004  18092.0  18145.0   
3507  1004  18178.0  18425.0   
3508  1004  18180.0  18399.0   
3509  1004  18412.0  18432.0   

                                                    cat           tier  \
0                                  questi piattini (RE)  Caregiver voc   
1                                 questi piattini. (RE)  Caregiver voc   
2                                 mescoli la pappa (DA)  Caregiver voc   
3                                mescoli la pappa. (DA)  Caregiver voc   
4     e gliene dai un pochino alla bambolina forse? ...  Caregiver voc   
...                                                 ...            ...   
3505           Guarda guarda dove passa (ride)\t(DA AL)  Caregiver voc   
3506           Guarda g